# PROJEKT MODELU REGRESYJNEGO PRZEWIDYWANIA CEN MIESZKAŃ NA PODSTAWIE #

# ALGORYTMU LGBM #

  ## KWIECIEŃ 2025 ##

In [1]:
import pandas as pd
import mlflow
# from pycaret.datasets import get_data 
from pycaret.regression import setup, pull, compare_models, plot_model, load_model, tune_model, finalize_model, save_model, predict_model, get_config
import pymysql
from sqlalchemy import create_engine
import numpy as np
# from scipy.stats import skewnorm 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
# from joblib import parallel_backend 

In [ ]:
MLFLOW_EXPERIMENT_NAME = 'Investoro_Ceny'
MLFLOW_TAGS = {'data': 'Investoro_ceny', 'library': 'pycaret'}

mlflow.set_tracking_uri("http://localhost:5000")

In [ ]:
# tę komórkę uruchom jeśli czerpiesz dane z pliku .csv
df_original  = pd.read_csv('data.csv', sep=',')

In [ ]:
df_original 

In [ ]:
df_original .head(10)

In [ ]:
df_original .sample(10)

In [ ]:
df_original.info()

In [ ]:
df_original [df_original .duplicated()]

In [ ]:
df_original .nunique()

In [ ]:
df_corr_temp = df_original.copy()
if pd.api.types.is_string_dtype(df_corr_temp['BuiltYear']):
    df_corr_temp['BuiltYear_Num'] = pd.to_datetime(df_corr_temp['BuiltYear'], format='%Y', errors='coerce').dt.year
elif pd.api.types.is_datetime64_any_dtype(df_corr_temp['BuiltYear']):
     df_corr_temp['BuiltYear_Num'] = df_corr_temp['BuiltYear'].dt.year
else:
    df_corr_temp['BuiltYear_Num'] = pd.to_numeric(df_corr_temp['BuiltYear'], errors='coerce') # Ostateczna próba

cols_for_corr = ['Area', 'Price', 'BuiltYear_Num', 'Floor', 'Floors', 'CommunityScore', 'CountyNumber', 'CommunityNumber',
                   'RegionNumber','KindNumber']
# Upewnij się, że wszystkie kolumny istnieją i są numeryczne
valid_cols_for_corr = [col for col in cols_for_corr if col in df_corr_temp.columns and pd.api.types.is_numeric_dtype(df_corr_temp[col])]
correlation_matrix = df_corr_temp[valid_cols_for_corr].corr()

In [ ]:
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Price')

In [ ]:
df_original .isnull()

In [ ]:
df_original .isnull().sum()

In [ ]:
df_cleaned = df_original.copy()
print(f"Rozmiar df_cleaned przed czyszczeniem: {df_cleaned.shape}")

df_cleaned.dropna(subset=['Area', 'Price', 'Location'], inplace=True)
print(f"Rozmiar df_cleaned po usunięciu NaN z Area, Price, Location: {df_cleaned.shape}")
display(df_cleaned.isnull().sum().sort_values(ascending=False).head(15))

In [ ]:
Q1_price = df_cleaned["Price"].quantile(0.25)
Q3_price = df_cleaned["Price"].quantile(0.75)
IQR_price = Q3_price - Q1_price
lower_bound_price = Q1_price - 1.5 * IQR_price
upper_bound_price = Q3_price + 1.5 * IQR_price
df_cleaned = df_cleaned[~((df_cleaned["Price"] < lower_bound_price) | (df_cleaned["Price"] > upper_bound_price))]
print(f"Rozmiar df_cleaned po usunięciu outlierów z Price: {df_cleaned.shape}")

In [ ]:
if "PricePerSquareMeter" in df_cleaned.columns and df_cleaned["PricePerSquareMeter"].isnull().sum() < len(df_cleaned) * 0.8: 
    df_cleaned.dropna(subset=['PricePerSquareMeter'], inplace=True) 
    Q1_ppsm = df_cleaned["PricePerSquareMeter"].quantile(0.25)
    Q3_ppsm = df_cleaned["PricePerSquareMeter"].quantile(0.75)
    IQR_ppsm = Q3_ppsm - Q1_ppsm
    lower_bound_ppsm = Q1_ppsm - 1.5 * IQR_ppsm
    upper_bound_ppsm = Q3_ppsm + 1.5 * IQR_ppsm
    df_cleaned = df_cleaned[~((df_cleaned["PricePerSquareMeter"] < lower_bound_ppsm) | (df_cleaned["PricePerSquareMeter"] > upper_bound_ppsm))]
    print(f"Rozmiar df_cleaned po usunięciu outlierów z PricePerSquareMeter: {df_cleaned.shape}")
else:
    print("Kolumna 'PricePerSquareMeter' nie użyta do usuwania outlierów (brak lub za dużo NaN).")

In [ ]:
Q1_area = df_cleaned["Area"].quantile(0.25)
Q3_area = df_cleaned["Area"].quantile(0.75)
IQR_area = Q3_area - Q1_area
lower_bound_area = Q1_area - 1.5 * IQR_area
upper_bound_area = Q3_area + 1.5 * IQR_area
df_cleaned = df_cleaned[~((df_cleaned["Area"] < lower_bound_area) | (df_cleaned["Area"] > upper_bound_area))]
print(f"Rozmiar df_cleaned po usunięciu outlierów z Area: {df_cleaned.shape}")

In [ ]:
df_cleaned['BuiltYear'] = pd.to_datetime(df_cleaned['BuiltYear'], format='%Y', errors='coerce')
print("Konwersja BuiltYear na datetime w df_cleaned zakończona.")

In [ ]:
print("Informacje o df_cleaned po wszystkich krokach czyszczenia:")
df_cleaned.info()
print("\nBraki danych w df_cleaned (%):")
display(df_cleaned.isnull().sum() / len(df_cleaned) * 100)
print("\nPierwsze wiersze df_cleaned:")
display(df_cleaned.head())

# Sprawdzenie braków - procentowo.

In [ ]:
print(f"Rozmiar df_cleaned przed podziałem na train/holdout: {df_cleaned.shape}")
train_df = df_cleaned.sample(frac=0.9, random_state=42)
holdout_df = df_cleaned.drop(train_df.index)

print(f"Rozmiar zbioru treningowego (train_df): {train_df.shape}")
print(f"Rozmiar zbioru holdout (holdout_df): {holdout_df.shape}")

In [ ]:
def convert_column_types(df_to_convert):
    df_copy = df_to_convert.copy()
    str_cols = ['VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber', 'RegionNumber', 'StreetNumber'] # Dodano StreetNumber
    for col in str_cols:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].astype(str)
    
    # BuiltYear powinno być już datetime, ale upewnijmy się
    if 'BuiltYear' in df_copy.columns and not pd.api.types.is_datetime64_any_dtype(df_copy['BuiltYear']):
         df_copy['BuiltYear'] = pd.to_datetime(df_copy['BuiltYear'], format='%Y', errors='coerce')
    return df_copy

train_df = convert_column_types(train_df)
holdout_df = convert_column_types(holdout_df)

print("\\nTypy danych w train_df po konwersji:")
train_df.info()
print("\\nTypy danych w holdout_df po konwersji:")
holdout_df.info()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

location_vectorizer = TfidfVectorizer(
    max_features=100, 
    stop_words=None,
    ngram_range=(1, 1),
    min_df=5,
    max_df=0.95
)

In [ ]:
print("Przetwarzanie TF-IDF dla zbioru treningowego...")
train_df_copy = train_df.copy() # Pracujemy na kopii
train_df_copy['Location_Clean'] = train_df_copy['Location'].fillna('').astype(str)
train_location_tfidf_features = location_vectorizer.fit_transform(train_df_copy['Location_Clean'])

try:
    feature_names = location_vectorizer.get_feature_names_out()
except AttributeError:
    feature_names = location_vectorizer.get_feature_names_() 
    
train_location_tfidf_df = pd.DataFrame(
    train_location_tfidf_features.toarray(),
    columns=['loc_tfidf_' + name for name in feature_names],
    index=train_df_copy.index # Ważne, aby zachować oryginalny indeks
)
print(f"Utworzono {train_location_tfidf_df.shape[1]} cech TF-IDF dla zbioru treningowego.")

train_df_processed = pd.concat(
    [train_df_copy.drop(columns=['Location', 'Location_Clean'], errors='ignore'), train_location_tfidf_df], 
    axis=1
)
# WAŻNE: Usuń wiersze gdzie 'Price' < 20000 (lub inna wartość) dopiero PO przetworzeniu TF-IDF, 
# aby uniknąć problemów z niedopasowaniem indeksów przy konkatenacji.
train_df_processed = train_df_processed[train_df_processed['Price'] >= 20000] 
print(f"Rozmiar train_df_processed po usunięciu cen < 20000: {train_df_processed.shape}")
display(train_df_processed.head())

In [ ]:
print("Przetwarzanie TF-IDF dla zbioru holdout...")
holdout_df_copy = holdout_df.copy() # Pracujemy na kopii
holdout_df_copy['Location_Clean'] = holdout_df_copy['Location'].fillna('').astype(str)
holdout_location_tfidf_features = location_vectorizer.transform(holdout_df_copy['Location_Clean']) # Użyj transform

holdout_location_tfidf_df = pd.DataFrame(
    holdout_location_tfidf_features.toarray(),
    columns=['loc_tfidf_' + name for name in feature_names],
    index=holdout_df_copy.index # Ważne, aby zachować oryginalny indeks
)
print(f"Utworzono {holdout_location_tfidf_df.shape[1]} cech TF-IDF dla zbioru holdout.")

holdout_df_processed = pd.concat(
    [holdout_df_copy.drop(columns=['Location', 'Location_Clean'], errors='ignore'), holdout_location_tfidf_df],
    axis=1
)
print(f"Rozmiar holdout_df_processed: {holdout_df_processed.shape}")
display(holdout_df_processed.head())

In [ ]:
categorical_features_initial = [
    'BuildingType', 'BuildingCondition', 'TypeOfMarket', 'OwnerType', 'Type', 'OfferFrom',
    'VoivodeshipNumber', 'CountyNumber', 'CommunityNumber', 'KindNumber', 'RegionNumber'
]
numeric_features_initial = [
    'Area', 'NumberOfRooms', 'Floor', 'Floors', 'CommunityScore'
]
date_features_initial = ['BuiltYear']

categorical_features_to_use = [col for col in categorical_features_initial if col in train_df_processed.columns]
numeric_features_to_use = [col for col in numeric_features_initial if col in train_df_processed.columns]
# Kolumny loc_tfidf_* zostaną dodane do numeric_features wewnątrz setup
date_features_to_use = [col for col in date_features_initial if col in train_df_processed.columns]

ignore_features_list_setup = [ # (...) lista jak w oryginalnym kodzie
    'SaleId', 'OriginalId', 'PortalId', 'Title', 'Description',
    'OfferPrice', 'RealPriceAfterRenovation', 'OriginalPrice',
    'PricePerSquareMeter', 'DateAddedToDatabase', 'DateAdded',
    'DateLastModification', 'DateLastRaises', 'NewestDate',
    'AvailableFrom', 'Link', 'Phone', 'MainImage', 'OtherImages',
    'NumberOfDuplicates', 'NumberOfRaises', 'NumberOfModifications',
    'IsDuplicatePriceLower', 'IsDuplicatePrivateOwner', 'Score', 'ScorePrecision',
    'NumberOfCommunityComments', 'NumberOfCommunityOpinions', 'Archive',
    'SubRegionNumber', 'EncryptedId',
    'StreetNumber' # StreetNumber jest teraz stringiem, jeśli ma za dużo wartości, można go tu dodać
]
ignore_features_final = [col for col in ignore_features_list_setup if col in train_df_processed.columns]

print("--- Informacje przed PyCaret setup ---")
print("Liczba kolumn w train_df_processed:", len(train_df_processed.columns.tolist()))
print("Cechy kategoryczne:", categorical_features_to_use)
print("Cechy numeryczne (początkowe):", numeric_features_to_use)
print("Cechy daty:", date_features_to_use)
print("Ignorowane cechy:", ignore_features_final)
print("------------------------------------")

In [ ]:
import os
# Utwórz dedykowany katalog dla tego testu, jeśli nie istnieje
current_directory = os.getcwd() 
local_mlruns_path = os.path.join(current_directory, "mlruns_DIRECT_LOCAL_TEST") 

if not os.path.exists(local_mlruns_path):
    os.makedirs(local_mlruns_path)
    print(f"Utworzono katalog: {local_mlruns_path}")
else:
    print(f"Katalog już istnieje: {local_mlruns_path}")

absolute_mlruns_path = os.path.abspath(local_mlruns_path)
tracking_uri = f"file:///{absolute_mlruns_path.replace(os.sep, '/')}"
mlflow.set_tracking_uri(tracking_uri)

print(f"MLflow tracking URI ustawione na: {mlflow.get_tracking_uri()}")

# MLFLOW_EXPERIMENT_NAME powinno być zdefiniowane wcześniej w Twoim notebooku
# np. MLFLOW_EXPERIMENT_NAME = 'Investoro_Ceny'

try:
    # Sprawdź, czy eksperyment istnieje
    experiment = mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
    if experiment is None:
        # Jeśli nie istnieje, stwórz go.
        # Dla logowania typu 'file://', MLflow sam zarządzi lokalizacją artefaktów
        # w podkatalogach struktury 'mlruns'.
        experiment_id = mlflow.create_experiment(MLFLOW_EXPERIMENT_NAME)
        print(f"Utworzono nowy eksperyment MLflow: '{MLFLOW_EXPERIMENT_NAME}' o ID: {experiment_id}")
    else:
        experiment_id = experiment.experiment_id
        print(f"Znaleziono istniejący eksperyment: '{MLFLOW_EXPERIMENT_NAME}' o ID: {experiment_id}")
    
    # Ustaw eksperyment jako aktywny
    mlflow.set_experiment(experiment_name=MLFLOW_EXPERIMENT_NAME)
    print(f"Aktywny eksperyment MLflow ustawiony na: '{MLFLOW_EXPERIMENT_NAME}'")

except Exception as e:
    print(f"Błąd podczas ustawiania/tworzenia eksperymentu MLflow: {e}")
    import traceback
    print(traceback.format_exc())


In [ ]:
reg_exp = None 
try:
    loc_tfidf_cols = [col for col in train_df_processed.columns if 'loc_tfidf_' in col]
    reg_exp = setup(
        data=train_df_processed, 
        target='Price',
        log_experiment=True,
        experiment_name=MLFLOW_EXPERIMENT_NAME,
        categorical_features=categorical_features_to_use,
        numeric_features=numeric_features_to_use + loc_tfidf_cols, # Dodajemy cechy TF-IDF
        date_features=date_features_to_use,
        ignore_features=ignore_features_final,
        # ... (reszta parametrów jak w oryginalnym kodzie) ...
    )
except Exception as e:
    print(f"Błąd podczas setup PyCaret: {e}")
    import traceback
    print(traceback.format_exc())

if reg_exp:
    best_model_from_compare = compare_models() # Zapisz najlepszy model z compare_models
    compare_metrics_df = pull()
    display(compare_metrics_df)
else:
    print("Nie udało się zainicjować eksperymentu PyCaret.")

In [ ]:
reg_exp.X_train_transformed.head()

In [ ]:
from pycaret.regression import get_config
import matplotlib.pyplot as plt
import seaborn as sns

# Wyciągnij dane oryginalne i przetransformowane
df_raw = train_df_processed.copy()
df_transformed = get_config("X_train").copy()
df_transformed["Price"] = get_config("y_train")

# Rysowanie wykresów
#fig, axes = plt.subplots(1, 2, figsize=(12, 6))

#sns.histplot(df_raw["Price"], ax=axes[0])
#axes[0].set_title("Raw Data")

#sns.histplot(df_transformed["Price"], ax=axes[1])
#axes[1].set_title("Transformed Data")

#plt.tight_layout()
#plt.show()

In [ ]:
reg_exp.dataset.head()

In [ ]:
reg_exp.dataset_transformed.head()

In [ ]:
#df_transformed.plot.scatter(x='Area', y='Price');

In [ ]:
#df_transformed.plot.scatter(x='BuiltYear', y='Price');

In [ ]:
reg_exp.plot_model(best_model_from_compare, plot='feature')

In [ ]:
if reg_exp and best_model_from_compare:
    print("Predykcja na zbiorze testowym (z podziału PyCaret) przy użyciu dostrojonego modelu:")
    predict_model(best_model_from_compare) # To wyświetli metryki na wewnętrznym zbiorze testowym
    test_set_metrics_after_tuning = pull()
    display(test_set_metrics_after_tuning)

In [ ]:
best_final_model = None
if reg_exp and best_model_from_compare:
    best_final_model = finalize_model(best_model_from_compare, experiment_custom_tags={"step": "final_tuned"})
    print("Sfinalizowany model (po strojeniuu):")
    display(best_final_model)
elif reg_exp and 'best_model_from_compare' in locals() and best_model_from_compare is not None:
    print("Strojenie nie powiodło się lub zostało pominięte. Finalizuję najlepszy model z compare_models.")
    best_final_model = finalize_model(best_model_from_compare, experiment_custom_tags={"step": "final_compare"})
    display(best_final_model)
else:
    print("Nie można sfinalizować modelu.")

In [ ]:
save_model(best_final_model, '0_full-basic-model')

In [ ]:
predict_model(best_final_model, data=holdout_df_processed)

In [ ]:
display(pull())

In [ ]:
final_holdout_predictions_df = None # Zmiana nazwy dla spójności ze skryptem
if reg_exp and best_final_model and 'holdout_df_processed' in locals() and not holdout_df_processed.empty:
    print("Predykcja na zbiorze holdout (za pomocą sfinalizowanego modelu)...")
    holdout_data_for_pred = holdout_df_processed.copy()
    
    # Dodanie prawdziwych cen do DataFrame z predykcjami dla łatwiejszej oceny (jeśli są dostępne)
    if 'Price' in holdout_data_for_pred.columns:
        holdout_data_for_pred = holdout_data_for_pred.drop(columns=['Price'])

    final_holdout_predictions_df = predict_model(best_final_model, data=holdout_data_for_pred)    

        # Można tu obliczyć metryki ręcznie lub użyć narzędzi sklearn
        from sklearn.metrics import r2_score, mean_absolute_error
        r2 = r2_score(final_holdout_predictions_df['Actual_Price'], final_holdout_predictions_df['prediction_label'])
        mae = mean_absolute_error(final_holdout_predictions_df['Actual_Price'], final_holdout_predictions_df['prediction_label'])
        print(f"Metryki na zbiorze holdout (sfinalizowany model): R2 = {r2:.4f}, MAE = {mae:.2f}")
        if mlflow.active_run():
             mlflow.log_metric("holdout_final_R2", r2)
             mlflow.log_metric("holdout_final_MAE", mae)


else:
    print("Nie można wykonać predykcji na zbiorze holdout - brak sfinalizowanego modelu lub danych.")

In [ ]:
if final_holdout_predictions_df is not None and 'holdout_df' in locals():
    # holdout_df to oryginalny zbiór holdout PRZED dodaniem cech TF-IDF i innymi transformacjami PyCaret
    # final_holdout_predictions_df ma predykcje i oryginalny indeks z holdout_df_processed

    # Przygotuj df_last: SaleId, oryginalna cena (Price), przewidziana cena (prediction_label)
    df_last = holdout_df[['SaleId', 'Price']].copy() # Używamy holdout_df dla oryginalnych wartości
    
    # Dodajemy kolumnę predykcji. Musimy upewnić się, że indeksy pasują.
    # predict_model zachowuje indeks oryginalnego DataFrame przekazanego do `data=`
    df_last['PredictedPrice_num'] = final_holdout_predictions_df.loc[df_last.index, 'prediction_label']

    # Formatowanie dla czytelności w CSV
    df_last['Price_formatted'] = df_last['Price'].apply(lambda x: f"{x:,.0f}" if pd.notnull(x) else None)
    df_last['PredictedPrice_formatted'] = df_last['PredictedPrice_num'].apply(lambda x: f"{x:,.0f}" if pd.notnull(x) else None)
    
    # Zapisz plik porównawczy z oryginalną ceną i przewidywaną
    df_last_to_save_compare = df_last[['SaleId', 'Price_formatted', 'PredictedPrice_formatted']].rename(
        columns={'Price_formatted': 'Original_Price', 'PredictedPrice_formatted': 'Predicted_Price'}
    )
    df_last_to_save_compare.to_csv('0_new_prices_compare.csv', index=False)
    print("Plik 0_new_prices_compare.csv został zapisany.")
    display(df_last_to_save_compare.head())

    # Zapisz pełny oryginalny zbiór holdout z dodaną kolumną predykcji
    holdout_df_with_predictions = holdout_df.copy()
    # Ponownie, dopasuj po indeksie
    holdout_df_with_predictions['PredictedPrice'] = final_holdout_predictions_df.loc[holdout_df_with_predictions.index, 'prediction_label']
    
    cols_to_save = list(holdout_df.columns) # Oryginalne kolumny
    cols_to_save.insert(cols_to_save.index('Price') + 1, 'PredictedPrice') # Wstaw PredictedPrice po Price
    # Usuń PredictedPrice z końca, jeśli już tam jest przez przypadek
    if 'PredictedPrice' in cols_to_save[:-1] and cols_to_save[-1] == 'PredictedPrice':
         cols_to_save.pop()
    
    holdout_df_with_predictions = holdout_df_with_predictions[cols_to_save]

    holdout_df_with_predictions.to_csv('full_holdout_with_predictions.csv', index=False)
    print("Plik full_holdout_with_predictions.csv został zapisany.")
    display(holdout_df_with_predictions.head())

else:
    print("Nie można zapisać wyników - brak predykcji (final_holdout_predictions_df) lub oryginalnego zbioru holdout (holdout_df).")

In [ ]:
# Komórka do przygotowania df_original_for_prediction
print("Przygotowywanie df_original do predykcji...")

df_original_for_prediction = df_original.copy()

df_original_for_prediction.dropna(subset=['Area', 'Location'], inplace=True)
print(f"Rozmiar po usunięciu NaN z Area, Location: {df_original_for_prediction.shape}")


if not df_original_for_prediction.empty: # Sprawdzenie czy DataFrame nie jest pusty
    Q1_area_orig = df_original_for_prediction["Area"].quantile(0.25)
    Q3_area_orig = df_original_for_prediction["Area"].quantile(0.75)
    IQR_area_orig = Q3_area_orig - Q1_area_orig
    lower_bound_area_orig = Q1_area_orig - 1.5 * IQR_area_orig
    upper_bound_area_orig = Q3_area_orig + 1.5 * IQR_area_orig
    df_original_for_prediction = df_original_for_prediction[
        ~((df_original_for_prediction["Area"] < lower_bound_area_orig) | (df_original_for_prediction["Area"] > upper_bound_area_orig))
    ]
    print(f"Rozmiar po usunięciu outlierów z Area: {df_original_for_prediction.shape}")
else:
    print("DataFrame jest pusty po usunięciu NaN, pomijam usuwanie outlierów.")

if not df_original_for_prediction.empty:
    df_original_for_prediction = convert_column_types(df_original_for_prediction) # Używamy funkcji convert_column_types
    print("Konwersja typów danych w df_original_for_prediction zakończona.")
else:
    print("DataFrame jest pusty, pomijam konwersję typów.")


if not df_original_for_prediction.empty:
    df_original_for_prediction['Location_Clean'] = df_original_for_prediction['Location'].fillna('').astype(str)
    original_location_tfidf_features = location_vectorizer.transform(df_original_for_prediction['Location_Clean'])

    try:
        feature_names_loc = location_vectorizer.get_feature_names_out()
    except AttributeError:
        feature_names_loc = location_vectorizer.get_feature_names_()

    original_location_tfidf_df = pd.DataFrame(
        original_location_tfidf_features.toarray(),
        columns=['loc_tfidf_' + name for name in feature_names_loc],
        index=df_original_for_prediction.index
    )

    df_original_processed = pd.concat(
        [df_original_for_prediction.drop(columns=['Location', 'Location_Clean'], errors='ignore'), original_location_tfidf_df],
        axis=1
    )
    print(f"Utworzono {original_location_tfidf_df.shape[1]} cech TF-IDF dla df_original_processed.")
    print(f"Rozmiar df_original_processed: {df_original_processed.shape}")
else:
    print("DataFrame jest pusty, pomijam przetwarzanie TF-IDF.")
    df_original_processed = pd.DataFrame() # Pusty DataFrame, aby uniknąć błędów później

display(df_original_processed.head() if not df_original_processed.empty else "DataFrame jest pusty")

In [ ]:
original_predictions = None # Inicjalizacja

if best_final_model and not df_original_processed.empty:
    print(f"Rozpoczynanie predykcji na df_original_processed o kształcie: {df_original_processed.shape}")
    
    data_for_prediction_final = df_original_processed.copy()
    
    # ZAPAMIĘTAJ oryginalne ceny, jeśli istnieją, zanim usuniesz kolumnę
    original_prices = None
    if 'Price' in data_for_prediction_final.columns:
        original_prices = data_for_prediction_final['Price'].copy() # Zapisujemy oryginalne ceny
        data_for_prediction_final = data_for_prediction_final.drop(columns=['Price']) # Usuwamy kolumnę Price
        print("Usunięto kolumnę 'Price' z danych przekazywanych do predykcji.")

    original_predictions = predict_model(best_final_model, data=data_for_prediction_final)
    
    print("\nPierwsze wiersze predykcji dla df_original_processed:")
    display(original_predictions.head())
    
    # Jeśli chcesz dodać z powrotem oryginalne ceny (dla porównania)
    if original_prices is not None:
        # Upewnij się, że indeksy pasują. original_predictions powinien mieć ten sam indeks co data_for_prediction_final
        original_predictions['Original_Price'] = original_prices.loc[original_predictions.index] 
        print("\nPierwsze wiersze predykcji wraz z oryginalną ceną (jeśli była dostępna):")
        display(original_predictions[['Original_Price', 'prediction_label']].head())
else:
    if not best_final_model:
        print("Model (best_final_model) nie został poprawnie załadowany lub sfinalizowany.")
    if df_original_processed.empty:
        print("DataFrame df_original_processed jest pusty, nie można wykonać predykcji.")

In [ ]:
# Komórka 43 (poprzednio 88) - ZAPIS WYNIKÓW (po Opcji 1 w predykcji)
if original_predictions is not None:
    # original_predictions ma indeksy z df_original_processed (po usunięciu NaN w Area/Location i outlierów w Area)
    # oraz kolumnę 'prediction_label'. Może też mieć inne kolumny z df_original_processed (bez Price).
    
    # Chcemy dodać 'prediction_label' do oryginalnego df_original
    
    # 1. Stwórz DataFrame tylko z SaleId (jeśli jest w indeksie original_predictions, zresetuj go)
    #    i prediction_label
    if original_predictions.index.name == 'SaleId': # Jeśli SaleId jest indeksem
        predictions_to_merge = original_predictions[['prediction_label']].reset_index()
    elif 'SaleId' in original_predictions.columns: # Jeśli SaleId jest kolumną
         predictions_to_merge = original_predictions[['SaleId', 'prediction_label']].copy()
    else: # Jeśli SaleId nie ma, a indeks jest numeryczny, musimy go odzyskać
        # To jest bardziej skomplikowane, jeśli SaleId nie było zachowane w df_original_processed
        # Załóżmy, że SaleId było w df_original_processed i zostało przeniesione do original_predictions
        # Jeśli nie, trzeba by wrócić do df_original_processed i wyciągnąć SaleId z niego po indeksie.
        # Dla uproszczenia zakładam, że SaleId jest w original_predictions
        print("OSTRZEŻENIE: Brak kolumny 'SaleId' w original_predictions. Łączenie może być niepoprawne.")
        # Spróbuj użyć indeksu, jeśli odpowiada SaleId z df_original_processed
        predictions_to_merge = pd.DataFrame({
            'SaleId': df_original_processed.loc[original_predictions.index, 'SaleId'], 
            'prediction_label': original_predictions['prediction_label']
        })

    predictions_to_merge.rename(columns={'prediction_label': 'PredictedPrice'}, inplace=True)
    
    # 2. Połącz z oryginalnym df_original
    df_original_with_all_predictions = pd.merge(
        df_original, # Używamy oryginalnego df_original
        predictions_to_merge,
        on='SaleId',
        how='left' 
    )
    
    # 3. Przenieś kolumnę 'PredictedPrice_LGBM'
    if 'Price' in df_original_with_all_predictions.columns and 'PredictedPrice' in df_original_with_all_predictions.columns:
        cols = list(df_original_with_all_predictions.columns)
        price_index = cols.index('Price')
        # Upewnij się, że 'PredictedPrice_LGBM' nie jest już w liście, zanim ją wstawisz
        if 'PredictedPrice' in cols:
            cols.remove('PredictedPrice')
        cols.insert(price_index + 1, 'PredictedPrice')
        df_original_with_all_predictions = df_original_with_all_predictions[cols]
        
    print("\nPierwsze wiersze df_original z dodanymi predykcjami:")
    display(df_original_with_all_predictions.head())
    
    # 4. Zapisz do CSV
    df_original_with_all_predictions.to_csv('sale_2024_0_predict.csv', index=False)
    print("\nPlik 'sale_2024_0_with_100_predictions.csv' został zapisany.")
    
    print("\nPrzykładowe wiersze z oryginalną ceną i predykcją (gdzie predykcja istnieje):")
    display(df_original_with_all_predictions[df_original_with_all_predictions['PredictedPrice'].notna()][['SaleId', 'Price', 'PredictedPrice']].head(20))
    
else:
    print("Brak predykcji do zapisania (original_predictions jest None).")

In [ ]:
df_original_with_all_predictions